## Enhancing Retrieval Augmented Generation for Documents with Integrated Images, Graphics and Tables

Implementing Retrieval Augmented Generation (RAG) presents unique challenges when working with documents rich in images and tables. Traditional RAG models excel with textual data but often falter when visual elements play a crucial role in conveying information. In this cookbook, we bridge that gap by leveraging vision models to extract and interpret visual content, ensuring that the generated responses are as informative and accurate as possible.

Our approach involves parsing documents into images and utilizing metadata tagging to identify pages containing images or tables. When a semantic search retrieves such a page, we pass the page image to a vision model instead of relying solely on text. This method enhances the model's ability to understand and answer user queries that pertain to visual data.

In this cookbook, we will explore and demonstrate the following key concepts:

##### 1. Setting Up a Vector Store with Pinecone:
- Learn how to initialize and configure Pinecone to store vector embeddings efficiently.

##### 2. Parsing PDFs and Extracting Visual Information:
- Discover techniques for converting PDF pages into images.
- Use vision models to detect pages with images or tables and extract pertinent information.
- Extract the information from images and tables to enable semantic search. 

##### 3. Generating Embeddings: 
- Flag the pages that have visual content so we can use this to setup meta-data filters on vector store. 
- Utilize embedding models to create vector representations of textual data.

#### 4. Uploading embeddings to Pinecone: 
- Upload these embeddings to Pinecone for seamless storage and retrieval. 

##### 5. Performing Semantic Search for Relevant Pages:
- Implement semantic search to find pages that best match the user's query. 

##### 6. Handling Pages with Visual Content:
- When metadata indicates the presence of an image or table, learn how to pass the image to a vision model for question answering for additional context. 
- Understand how this process improves the accuracy of responses involving visual data.

 
By the end of this cookbook, you will have a robust understanding of how to implement RAG systems capable of processing and interpreting documents with complex visual elements. This knowledge will empower you to build AI solutions that deliver richer, more accurate information, enhancing user satisfaction and engagement.

We will use the World Bank report - [A better bank for a better world: Annual Report 2024](https://documents1.worldbank.org/curated/en/099101824180532047/pdf/BOSIB13bdde89d07f1b3711dd8e86adb477.pdf) to illustrate the concepts as this document has a mix of images, tables and graphics data. 

### 1. Setting up a Vector Store with Pinecone 
In this section, we'll set up a vector store using Pinecone to store and manage our embeddings efficiently. Pinecone is a powerful vector database optimized for handling high-dimensional vector data, which is essential for tasks like semantic search and similarity matching.

**Prerequisites** 
Sign-up for Pinecone and obtain an API key by following the instructions here [Pinecone Database Quickstart](https://docs.pinecone.io/guides/get-started/quickstart)
Install the Pinecone SDK using `pip install "pinecone[grpc]"`. gRPC (gRPC Remote Procedure Call) is a high-performance, open-source universal RPC framework that uses HTTP/2 for transport, Protocol Buffers (protobuf) as the interface definition language, and enables client-server communication in a distributed system. It is designed to make inter-service communication more efficient and suitable for microservices architectures. 

**Storing the API Key Securely**
Store the API key in an .env file for security purposes in you project directory as follows:  
 `PINECONE_API_KEY=your-api-key-here`. Install `pip install python-dotenv` to read the API Key from the .env file. 

**Creating the Pinecone Index** 
We'll use the `create_index` function to initialize our embeddings database on Pinecone. There are two crucial parameters to consider:

1. Dimension: This must match the dimensionality of the embeddings produced by your chosen model. For example, OpenAI's text-embedding-ada-002 model produces embeddings with 1536 dimensions, while text-embedding-3-large produces embeddings with 3072 dimensions. We'll use the text-embedding-3-large model, so we'll set the dimension to 3072.

2. Metric: The distance metric determines how similarity is calculated between vectors. Pinecone supports several metrics, including cosine, dotproduct, and euclidean. For this cookbook, we'll use the cosine similarity metric. You can learn more about distance metrics in the [Pinecone Distance Metrics documentation](https://docs.pinecone.io/guides/indexes/understanding-indexes#distance-metrics).


In [1]:
import os
import time
# Import the Pinecone library
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec

from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv("PINECONE_API_KEY")

# Initialize a Pinecone client with your API key
pc = Pinecone(api_key)

# Create a serverless index
index_name = "my-test-index"

if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=3072,
        metric="cosine",
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )

# Wait for the index to be ready
while not pc.describe_index(index_name).status['ready']:
    time.sleep(1)


PineconeConfigurationError: You haven't specified an Api-Key.

Navigate to Indexes list on [Pinecone](https://app.pinecone.io/) and you should see your index listed there.

### 2. Parsing PDFs and Extracting Visual Information:

In this section, we will parse a PDF document and extract textual and visual information, such as text, images, and tables. The process involves three main steps:

1. Parse the PDF into individual pages: We split the PDF into separate pages for easier processing.
2. Convert PDF pages to images: This enabled vision GPT-4o vision capability to analyze the page as an image.  
3. Process images and tables: Provide instructions to GPT-4o to extract text, and also describe the images or tables in the document. 

**Prerequisites**

Before proceeding, make sure you have the following packages installed. Also ensure your OpenAI API key is setup as an environment variable. 

`pip install PyPDF2 pdf2image pytesseract pandas tqdm`

You may also need to install Poppler for PDF rendering. 
 
**Steps Breakdown:**

**1. Downloading and Chunking the PDF:**  
- The `chunk_document` function downloads the PDF from the provided URL and splits it into individual pages using PyPDF2.
- Each page is stored as a separate PDF byte stream in a list.

**2. Converting PDF Pages to Images:** 
- The `convert_page_to_image` function takes the PDF bytes of a single page and converts it into an image using pdf2image.
- The image is saved locally in an 'images' directory for further processing.

**3. Extracting Text Using OCR:**
- The `extract_text_from_image` function uses GPT-4o vision capability to extract text from the image of the page.
- This method can extract textual information even from scanned documents.

**4. Processing the Entire Document:** 
- The process_document function orchestrates the processing of each page.
- It uses a progress bar (tqdm) to show the processing status.
- The extracted information from each page is collected into a list and then converted into a Pandas DataFrame.


In [23]:
import base64
import requests
import os
import pandas as pd
from PyPDF2 import PdfReader, PdfWriter
from pdf2image import convert_from_bytes
from io import BytesIO
from openai import OpenAI
from tqdm import tqdm  # Import tqdm

document_to_parse = "https://documents1.worldbank.org/curated/en/099101824180532047/pdf/BOSIB13bdde89d07f1b3711dd8e86adb477.pdf"


def chunk_document(document_url):
    # Download the PDF document
    response = requests.get(document_url)
    pdf_data = response.content

    # Read the PDF data using PyPDF2
    pdf_reader = PdfReader(BytesIO(pdf_data))
    page_chunks = []

    for page_number, page in enumerate(pdf_reader.pages, start=1):
        pdf_writer = PdfWriter()
        pdf_writer.add_page(page)
        pdf_bytes_io = BytesIO()
        pdf_writer.write(pdf_bytes_io)
        pdf_bytes_io.seek(0)
        pdf_bytes = pdf_bytes_io.read()
        page_chunk = {
            'pageNumber': page_number,
            'pdfBytes': pdf_bytes
        }
        page_chunks.append(page_chunk)

    return page_chunks


def convert_page_to_image(pdf_bytes, page_number):
    # Convert the PDF page to an image
    images = convert_from_bytes(pdf_bytes)
    image = images[0]  # There should be only one page

    # Define the directory to save images (relative to your script)
    images_dir = 'images'  # Use relative path here

    # Ensure the directory exists
    os.makedirs(images_dir, exist_ok=True)

    # Save the image to the images directory
    image_file_name = f"page_{page_number}.png"
    image_file_path = os.path.join(images_dir, image_file_name)
    image.save(image_file_path, 'PNG')

    # Return the relative image path
    return image_file_path


def get_vision_response(prompt, image_path):
    client = OpenAI()

    # Function to encode the image
    def encode_image(local_image_path):
        with open(local_image_path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')

    # Getting the base64 string
    base64_image = encode_image(image_path)

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{base64_image}"
                        },
                    },
                ],
            }
        ],
    )
    return response


def process_document(document_url):
    try:
        # Update document status to 'Processing'
        print("Document processing started")

        # Get per-page chunks
        page_chunks = chunk_document(document_url)
        total_pages = len(page_chunks)

        # Prepare a list to collect page data
        page_data_list = []

        # Add progress bar here
        for page_chunk in tqdm(page_chunks, total=total_pages, desc='Processing Pages'):
            page_number = page_chunk['pageNumber']
            pdf_bytes = page_chunk['pdfBytes']

            # Convert page to image
            image_path = convert_page_to_image(pdf_bytes, page_number)

            # Prepare question for vision API
            system_prompt = (
                "The user will provide you an image of a document file. Perform the following actions: "
                "1. Transcribe the text on the page. **TRANSCRIPTION OF THE TEXT:**"
                "2. If there is a chart, describe the image and include the text **DESCRIPTION OF THE IMAGE OR CHART**"
                "3. If there is a table, transcribe the table and include the text **TRANSCRIPTION OF THE TABLE**"
            )

            # Get vision API response
            vision_response = get_vision_response(system_prompt, image_path)

            # Extract text from vision response
            text = vision_response.choices[0].message.content

            # Collect page data
            page_data = {
                'PageNumber': page_number,
                'ImagePath': image_path,
                'PageText': text
            }
            page_data_list.append(page_data)

        # Create DataFrame from page data
        pdf_df = pd.DataFrame(page_data_list)
        print("Document processing completed.")
        print("DataFrame created with page data.")

        # Return the DataFrame
        return pdf_df

    except Exception as err:
        print(f"Error processing document: {err}")
        # Update document status to 'Error'


df = process_document(document_to_parse)

Document processing started


Processing Pages: 100%|██████████| 49/49 [14:39<00:00, 17.95s/it]

Document processing completed.
DataFrame created with page data.


Let's examine the dataframe to make sure the pages have been processed. You should also be able to see page images generated under directory 'images'. 

In [24]:
from IPython.display import display, HTML

# Convert the DataFrame to an HTML table and display it
display(HTML(df.to_html()))


Let's examine a page that has embedded images and text. We can notice that the vision modality extracted the visual information effectively.  
 
![Page-21](./images/page_21.png)

In [25]:
# Filter and print rows where pageNumber is 21
filtered_rows = df[df['PageNumber'] == 21]
print(filtered_rows.PageText)

20    **TRANSCRIPTION OF THE TEXT:**\n\nWe also committed $35 million in grants to support emergency relief in Gaza. Working with the World Food Programme, the World Health Organization, and the UN Children’s Fund, the grants support the delivery of emergency food, water, and medical supplies. In the West Bank, we approved a $13 million grant for the continuation of education for children, $22 million to support municipal services, and $45 million to strengthen healthcare and hospital services.\n\nEnabling green and inclusive growth\nTo help policymakers in the region advance their climate change and development goals, we published Country Climate and Development Reports for the West Bank and Gaza, Lebanon, and Tunisia. In Libya, the catastrophic flooding in September 2023 devastated eastern localities, particularly the city of Derna. The World Bank, together with the UN and the European Union, produced a Rapid Damage and Needs Assessment to inform recovery and reconstruction efforts.\

### 3. Generating Embeddings: 

In this section, we focus on transforming the textual content extracted from each page of the document into vector embeddings. These embeddings capture the semantic meaning of the text, enabling efficient similarity searches and various NLP tasks. We also identify pages containing visual elements, such as images or tables, and flag them for special handling.

**Adding a flag for visual content**  
  
To address pages with visual information, we leverage a vision model to extract content from charts, tables, and images. Using specific instructions, we tag the extracted text and flag pages containing visual content. Although the vision model effectively captures much of the visual data, some nuances may be lost in translation.

When a semantic search retrieves a page flagged for visual content, we use this tag to pass the original page as input to the model for Retrieval-Augmented Content Generation. A simple lambda function checks if `PageText` contains markers like `DESCRIPTION OF THE IMAGE OR CHART` or `TRANSCRIPTION OF THE TABLE`. If a marker is detected, the flag is set to 'Y'; otherwise, it's 'N'.

**Generating Embeddings with OpenAI's Embedding Model**  

We use OpenAI's embedding model, `text-embedding-3-large`, to generate high-dimensional embeddings that represent the semantic content of each page. 

<span style="bold">Note: It is crucial to ensure that the dimensions of the embedding model you use are consistent with the configuration of your Pinecone vector store. In our case, we setup the Pinecone database with 3072 dimensions to match the default dimensions of `text-embedding-3-large`</span> 


In [26]:
# Add a column to flag pages with visual content
df['Visual_Input_Needed'] = df['PageText'].apply(
    lambda x: 'Y' if '**DESCRIPTION OF THE IMAGE OR CHART**' in x or '**TRANSCRIPTION OF THE TABLE**' in x else 'N'
)


# Function to get embeddings
def get_embedding(text_input):
    client = OpenAI()

    response = client.embeddings.create(
        input=text_input,
        model="text-embedding-3-large"
    )
    return response.data[0].embedding


# Generate embeddings with a progress bar to display progress
embeddings = []
for text in tqdm(df['PageText'], desc='Generating Embeddings'):
    embedding = get_embedding(text)
    embeddings.append(embedding)

# Add the embeddings to the DataFrame
df['Embeddings'] = embeddings

Generating Embeddings: 100%|██████████| 49/49 [00:29<00:00,  1.67it/s]


Let's make sure both columns have been added successfully. We can check that page-21 that we examined has a Visual_Input_Needed flag set to "Y".  

In [27]:
display(HTML(df.to_html()))

#### 4. Uploading embeddings to Pinecone: 

We'll create meta-data tags for the data we've gathered on the pages, and upload the embeddings to Pinecone with the relevant meta-data tags, including the page text. 

In [29]:
import os
from tqdm import tqdm
import asyncio  # Import asyncio for asynchronous operations

# Assuming 'pc', 'index_name', and 'df' are defined elsewhere in your code
index = pc.Index(index_name)
document_id = 'WB_Report'

# Define the async function correctly
def upsert_vector(identifier, embedding, metadata):
    try:
        index.upsert([
            {
                'id': identifier,
                'values': embedding,
                'metadata': metadata
            }
        ])
    except Exception as e:
        print(f"Error upserting vector with ID {identifier}: {e}")
        raise


for idx, row in tqdm(df.iterrows(), total=df.shape[0], desc='Uploading to Pinecone'):
    pageNumber = row['PageNumber']
    
    # Create meta-data tags to be added to Pinecone 
    metadata = {
        'pageId': f"{document_id}-{pageNumber}",
        'pageNumber': pageNumber,
        'text': row['PageText'],
        'ImagePath': row['ImagePath'],
        'GraphicIncluded': row['Visual_Input_Needed']
    }

    upsert_vector(metadata['pageId'] , row['Embeddings'], metadata)  
    

Uploading to Pinecone: 100%|██████████| 49/49 [00:07<00:00,  6.37it/s]


Navigate to Indexes list on [Pinecone](https://app.pinecone.io/) and you should be able to view the vectors upserted into the database with metadata.


### 5. Performing Semantic Search for Relevant Pages:
- Implement semantic search to find pages that best match the user's query, regardless of whether the content is text or visual.

In [34]:
import json

def get_response_to_question(user_question, pc_index):
    # Get embedding of the question to find the relevant page with the information 
    question_embedding = get_embedding(user_question) 

    # get response vector embeddings 
    response = pc_index.query(
            vector=question_embedding,
            top_k=2,
            include_values=True,
            include_metadata=True
        )
    
    # Collect the metadata from the matches
    context_metadata = [match['metadata'] for match in response['matches']]
    
    # Convert the list of metadata dictionaries to a JSON string
    context_json = json.dumps(context_metadata, indent=2)
    
    prompt = f"""You are a helpful assistant. Use the following context and images to answer the question. In the answer, include the reference to the document, and page number you found the information on between <source></source> tags. If you don't find the information, you can say "I couldn't find the information"

    question: {user_question}
    
    <SOURCES>
    {context_json}
    </SOURCES>
    """
    
    client = OpenAI()
    
    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": prompt}
        ]
    )
    
    return completion.choices[0].message.content

question = "How many people did World bank help feed?"
answer = get_response_to_question(question, index)

print(answer)


The World Bank helped feed 156 million people in the first half of fiscal 2024. 

<source>WB_Report-4, page 4</source>


Now, let's ask a question where the information was included in a graphic representation. 

In [35]:
question = "What percentage was allocated to social protections in Western and Central Africa?"
answer = get_response_to_question(question, index)

print(answer)

The percentage allocated to social protections in Western and Central Africa was 8% in fiscal 2024. <source>WB_Report-13</source>


Make it a bit harder for RAG, let's ask a question that requires visual understanding of a table.  

In [37]:
question = "What was the increase in access to electricity between 2000 and 2012 in Western and Central Africa?"
answer = get_response_to_question(question, index)

print(answer)

The increase in access to electricity in Western and Central Africa between 2000 and 2012 was from 34.1% to 44.1%, making an increase of 10 percentage points. 

<source>WB_Report-13, page number 13</source>


This worked well. However, there may be instances where the information is contained with in images or graphics that may lose fidelity when translated to text. This can be the case for example for documents with complex engineering drawings. With the new vision modality, we can pass the image of the page as context to the model. In next section, we will examine how to enhance the accuracy of model response with image inputs. 

### 6. Handling Pages with Visual Content:
When metadata indicates the presence of an image or table, with the new vision modality, we can pass the image for the response. In this approach, we identify the right page using the vector embedding, however instead of passing the extracted text, we pass the image of the page as context for the model to respond to the question.  

In [38]:
import base64
import json

def get_response_to_question_with_images(user_question, pc_index):
    # Get embedding of the question to find the relevant page with the information 
    question_embedding = get_embedding(user_question) 

    # Get response vector embeddings 
    response = pc_index.query(
        vector=question_embedding,
        top_k=2,
        include_values=True,
        include_metadata=True
    )
    
    # Collect the metadata from the matches
    context_metadata = [match['metadata'] for match in response['matches']]

    # Function to encode the image
    def encode_image(local_image_path):
        with open(local_image_path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')
    
    # Build the message content
    message_content = []

    # Add the initial prompt
    initial_prompt = f"""You are a helpful assistant. Use the following images to answer the question. In the answer, include the reference to the page number or title of the section you found on the image. If you don't find the information, you can say "I couldn't find the information"

    question: {user_question}
    """
    
    message_content.append({"type": "text", "text": initial_prompt})

    # Process each metadata item to include text and images
    for metadata in context_metadata:
        image_path = metadata.get('ImagePath', None)
        
        try:
            base64_image = encode_image(image_path)
            image_type = 'jpeg'  # Adjust if your images are in a different format
            message_content.append({
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/{image_type};base64,{base64_image}"
                },
            })
        except Exception as e:
            print(f"Error encoding image at {image_path}: {e}")

    # Prepare the messages for the API call
    messages = [
        {
            "role": "user",
            "content": message_content
        }
    ]
    
    client = OpenAI()
    
    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=messages
    )
    
    return completion.choices[0].message.content


In [40]:
question = "How many people did World bank help feed?"
answer = get_response_to_question_with_images(question, index)

print(answer)

The World Bank helped feed 156 million people. This information is found in the section titled "DRIVING ACTION, MEASURING RESULTS" on page 4 of the World Bank Annual Report 2024.


In [43]:
question = "What percentage was allocated to social protections in Western and Central Africa?"
answer = get_response_to_question_with_images(question, index)

print(answer)

The percentage allocated to social protections in Western and Central Africa is 8%. This information is found in Figure 2: Western and Central Africa on page 22.


In [44]:
question = "What was the increase in access to electricity between 2000 and 2012 in Western and Central Africa?"
answer = get_response_to_question_with_images(question, index)

print(answer)

The increase in access to electricity between 2000 and 2012 in Western and Central Africa was from 34.1% to 44.1%.

(Source: Table 5: Western and Central Africa Regional Snapshot, page 23)


### Conclusion

In this cookbook, we embarked on a journey to enhance Retrieval-Augmented Generation (RAG) systems for documents rich in images, graphics and tables. Traditional RAG models, while proficient with textual data, often overlook the wealth of information conveyed through visual elements. By integrating vision models and leveraging metadata tagging, we've bridged this gap, enabling AI to interpret and utilize visual content effectively.

We began by setting up a vector store using Pinecone, establishing a foundation for efficient storage and retrieval of vector embeddings. Parsing PDFs and extracting visual information allowed us to convert document pages into images and identify those containing crucial visual data. By generating embeddings and flagging pages with visual content, we created a robust metadata filtering system within our vector store.

Uploading these embeddings to Pinecone facilitated seamless integration with our RAG system. Through semantic search, we retrieved relevant pages that matched user queries, ensuring that both textual and visual information were considered. Handling pages with visual content by passing them to vision models enhanced the accuracy and depth of the responses, particularly for queries dependent on images or tables.

Using the World Bank's **A Better Bank for a Better World: Annual Report 2024** as our guiding example, we demonstrated how these techniques come together to process and interpret complex documents. This approach not only enriches the information provided to users but also significantly enhances user satisfaction and engagement by delivering more comprehensive and accurate responses.

By following the concepts outlined in this cookbook, you are now equipped to build RAG systems capable of processing and interpreting documents with intricate visual elements. This advancement opens up new possibilities for AI applications across various domains where visual data plays a pivotal role.